# STEPS TO FOLLOW 

- Pick the Active Elibile Tables from config Table where active_flag=1 and load_flag=1
- And bq_to_gcs_status='COMPLETED'
- Reads the Latest PARQUET from GCS (gcs_path/dt=/*.parquet)--> uses the Latest dt Folder
- Writes to the Delta bronze path
- Create the Table From bronze path top bronze Dataset
- Update the config table

In [0]:
dbutils.widgets.text("mysql_host","136.113.73.13")
mysql_host=dbutils.widgets.get("mysql_host")

dbutils.widgets.text("mysql_port","3306")
mysql_port=dbutils.widgets.get("mysql_port")

dbutils.widgets.text("mysql_user","root")
mysql_user=dbutils.widgets.get("mysql_user")

dbutils.widgets.text("mysql_db","GCPMigrationMeta")
mysql_db=dbutils.widgets.get("mysql_db")

dbutils.widgets.text("mysql_password","Admin@1234")
mysql_password=dbutils.widgets.get("mysql_password")

dbutils.widgets.text("bq_sa_key","/Workspace/Users/anurag.srivastava@koantekorg.onmicrosoft.com/Migrationproject/datamigrationproject-475415-ffc1c7e9a773.json")
GOOGLE_APPLICATION_CREDENTIALS=dbutils.widgets.get("bq_sa_key")

dbutils.widgets.text("hive_db","bronze")
hive_db=dbutils.widgets.get("hive_db")

In [0]:
%pip install -q mysql-connector-python google-cloud-bigquery google-cloud-storage

In [0]:
import os, re, shutil, json
import mysql.connector as mc
from contextlib import contextmanager
from urllib.parse import urlparse
from google.cloud import storage
from pyspark.sql.functions import col, to_timestamp, to_date
from pyspark.sql.types import TimestampType

#Enclosing it in Dictiornary
MYSQL={
    'host':mysql_host.strip(),
    'port':int(mysql_port),
    'user':mysql_user.strip(),
    'pwd':mysql_password,
    'db':mysql_db
}

#Repalcing the dbfs:/ with /dbfs/`
BQ_KEY=GOOGLE_APPLICATION_CREDENTIALS.replace('dbfs:/','/dbfs/')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BQ_KEY  

In [0]:
@contextmanager
def mysql_conn():
    conn=mc.connect(
            host=MYSQL['host'],port=MYSQL['port'],
            user=MYSQL['user'],password=MYSQL['pwd'],database=MYSQL['db']
    )
    try:
        yield conn 
    except Exception as e:
        print(f"Failed to Connect to MYSQL \nHere is the Error -: \n{e}")
    finally:
        conn.close()



In [0]:
def fetch_eligible_tables():
    """Get rows to process with needed columns."""
    with mysql_conn() as conn:
        cur = conn.cursor(dictionary=True)
        cur.execute("""
          SELECT table_name, source_project, source_dataset, gcs_path
          FROM config_table
          WHERE active_flag=1 AND load_flag=1
          ORDER BY table_name
        """)
        rows = cur.fetchall()  # Get all matching rows as a list of dicts
        cur.close()
    return rows


In [0]:
def set_bronze_status(table_name,status,err=None):
    
    with mysql_conn() as conn:
        cur = conn.cursor()
        if status == "IN_PROGRESS":
            # Mark as in progress and clear any previous error
            cur.execute("""UPDATE GCPMigrationMeta.config_table 
                        SET gcs_to_bronze_status="IN_PROGRESS",last_run_ts=NOW(),error_message=NULL
                        where table_name=%s""",(table_name))            

        elif status == "COMPLETED":
            # Mark as in progress and clear any previous error
            cur.execute("""UPDATE GCPMigrationMeta.config_table 
                        SET gcs_to_bronze_status="COMPLETED",last_run_ts=NOW(),error_message=NULL
                        where table_name=%s""",(table_name))   
        
        else:  # FAILED
            # Mark as failed and store the error message (up to 2000 chars)
            cur.execute("""
              UPDATE config_table
                 SET bq_to_gcs_status='FAILED',
                     error_message=%s
               WHERE table_name=%s
            """, (str(err)[:2000] if err else "FAILED", table_name))
        conn.commit()
        cur.close()
